In [3]:
#Handle Imports
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os
import json
import pprint
import io
import requests
from PIL import Image
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import uuid
from ollama import chat
from ollama import ChatResponse
from random import randrange
import os
import glob
import shutil
import textwrap


#load from .env file which should exist
load_dotenv()

#initialize spotipy client
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)


In [4]:
#given an image performs the processing
def clear_images():
   for hgx in glob.glob("/Users/MurphyD1/Documents/Python/sloppa/images/*"):
        #print(hgx)
        os.remove(hgx)
   for hgx in glob.glob("/Users/MurphyD1/Documents/Python/sloppa/final_images/*"):
        #print(hgx)
        os.remove(hgx)
    

def slop_generator(track, artist, star):
    response: ChatResponse = chat(model='llama3.2', messages=[
      {
        'role': 'user',
        'content': f"""Generate some vague but smart-sounding commentary on the track {track} by {artist}. Should be indicative of a {star} star review. 20 words max and should sound like it was written by a relatively young person""",
      },
    ])
    return response['message']['content']


def download_image(link):
    url = link
    response = requests.get(url)
    file_path = str(uuid.uuid4()) + ".jpg"
    if response.status_code == 200:
        with open("images/" + file_path, 'wb') as file:
            file.write(response.content)
    return file_path



#given a playlist builds a dictionary of items 

def load_items(playlist):
    track_list = []
    result = sp.playlist(playlist, fields=None, market=None, additional_types=('track',))
    tracks = result['tracks']['items']
    tracks = tracks[0:5]
    
    for track in tracks:
        
        #extract track information 
        track = track['track']
        name = track['name']
        artists = track['artists']
        image = track['album']['images'][0]['url']
        artist  = (artists[0]['name'])
        
        #download image
        res = download_image(image)

        #assign random star rating and generate text slop for it
        stars = randrange(1,5)
        slop = slop_generator(name, artist, stars)

        #print(name, artist, image)
        d = {
            "name": name,
            "artist": artist,
            "image": image,
            "image_id": res,
            "analysis": slop,
            "stars": stars
        }
        track_list.append(d)

        #res = download_image(image)
    return track_list  

In [5]:
#given an image performs the processing
#TODO GET RID OF ALL THESE HARDCODED VALUES BEFORE I LOSE MY mIND :DD
def process_item(obj):
    
    STAR_SIZE = 65
    CENTER_IMG_SIZE = 1080
    TT_WIDTH = 1080
    TT_HEIGHT = 1920
    TITLE_SIZE = 62
    SUBTITLE_SIZE = 52
    PARAGRAPH_SIZE = 32
    TITLE_DIVIDER_SIZE = 31
    STAR_DIVIDER_SIZE = 75
    
    
    #Gather File Information
    track_title = obj['name']
    track_artist = obj['artist']
    image_id = obj['image_id']
    stars = obj['stars']
    slop = obj['analysis']
    file_path = f"""images/{image_id}"""
    
    #open and adjust star images
    star_f = Image.open('fill.png')
    star_u = Image.open('unfilled.png')
    star_f = star_f.resize((STAR_SIZE,STAR_SIZE), Image.LANCZOS)
    star_u = star_u.resize((STAR_SIZE,STAR_SIZE), Image.LANCZOS)

    #open and resize track image
    original_img = Image.open(file_path)
    resized_img = original_img.resize((CENTER_IMG_SIZE,CENTER_IMG_SIZE), Image.LANCZOS)
    
    #create a new blank image
    new_width, new_height = TT_WIDTH,TT_HEIGHT
    color = (166,123,91)
    background = Image.new("RGBA", (new_width, new_height), color)
    
    #paste track image onto background
    x_offset = 0
    y_offset = (TT_HEIGHT - TT_WIDTH) // 2
    background.paste(resized_img, (x_offset, y_offset))

    #image will be saved in PNG format since it's transparent, higher quality.
    png_format = image_id.replace(".jpg", ".png")
    new_filename = f"""final_images/{png_format}"""

    #initialize draw object to paste stars/text
    draw = ImageDraw.Draw(background)

    #load required fonts
    title_font = ImageFont.truetype("SourceCodePro-Bold.ttf",TITLE_SIZE)
    subtitle_font = ImageFont.truetype("SourceCodePro-SemiBold.ttf",SUBTITLE_SIZE)
    paragraph_font = ImageFont.truetype("SourceCodePro-Medium.ttf", PARAGRAPH_SIZE)
    
    title_text = f"""{track_artist}"""
    subtitle_text = f"""{track_title}"""
    paragraph_text = slop

    n_width = TT_WIDTH
    n_height = (TT_HEIGHT - CENTER_IMG_SIZE)/2
    n_height_b = CENTER_IMG_SIZE + n_height

    
    draw.text((n_width/2, n_height/2 - TITLE_DIVIDER_SIZE), title_text, font=title_font, anchor="mm", fill=(0,0,0,255))
    draw.text((n_width/2, n_height/2 + TITLE_DIVIDER_SIZE), subtitle_text, font=subtitle_font, anchor="mm", fill=(0,0,0,255))

    h3 = int(n_height/2 + STAR_DIVIDER_SIZE)
    w3  = int(n_width/2 - STAR_SIZE * 2.5)
    #print(w3)

    #layer = 
    for x in range(0,5):
        if x < stars:
            background.paste(star_f, (w3, h3), mask=star_f) 
        else:
            background.paste(star_u, (w3, h3), mask=star_f) 
        #background.paste(star_f, (w3, h3))
        w3 = w3 + 65
    
    #write out individual lines from the generated text
    lines = textwrap.wrap(paragraph_text, width=40)
    number_lines = len(lines)
    starting_height = TT_HEIGHT - n_height + 50
    for line in lines:
        draw.text((n_width/2, starting_height), line, font=paragraph_font, anchor="mm", fill=(0,0,0,255))
        starting_height = starting_height + PARAGRAPH_SIZE

    
    background.save(new_filename)

    background.close()
    original_img.close()
    star_f.close()
    star_u.close()


In [6]:
clear_images()
items = load_items('spotify:playlist:0VLYIYEEt27T8cPc2EGogY')

In [7]:
#process_item(items[0])

In [8]:
for item in items:
    process_item(item)